In [1]:
#trying to implement ensemble method
#https://datascience.stackexchange.com/questions/27169/taking-average-of-multiple-neural-networks
#mixture of experts 'with kmeans'
#https://en.wikipedia.org/wiki/Mixture_of_experts
#combining models together university of Tartu
#https://courses.cs.ut.ee/MTAT.03.277/2014_fall/uploads/Main/deep-learning-lecture-9-combining-multiple-neural-networks-to-improve-generalization-andres-viikmaa.pdf
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import time
#import keras
#from keras.utils import to_categorical

(trX, trY), (teX, teY) = tf.keras.datasets.fashion_mnist.load_data()
trX = trX.reshape(60000, 784)
teX = teX.reshape(10000, 784)

#trY = to_categorical(trY)
#teY = to_categorical(teY)

#np.save('one_hot_train_labels.npy', trY)
#np.save('one_hot_test_labels.npy', teY)

trY = np.load('one_hot_train_labels.npy')
teY = np.load('one_hot_test_labels.npy')


print("x_train shape:", trX.shape, "y_train shape:", trY.shape)
print("x_test shape:", teX.shape, "y_test shape:", teY.shape)

x_train shape: (60000, 784) y_train shape: (60000, 10)
x_test shape: (10000, 784) y_test shape: (10000, 10)


In [2]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))


def model(X, w_h1, w_h2, w_o):
    h1 = tf.nn.sigmoid(tf.matmul(X, w_h1)) # this is a basic mlp, think 2 stacked logistic regressions
    h = tf.nn.sigmoid(tf.matmul(h1, w_h2))
    #return tf.matmul(h, w_o, name="insertname_here") if we need to use names and save the models
    return tf.matmul(h, w_o) # note that we dont take the softmax at the end because our cost fn does that for us

#taken from https://stackoverflow.com/questions/47518609/for-loop-range-and-interval-how-to-include-last-step
def myRange(start,end,step):
    i = start
    while i < end:
        yield i
        i += step
    yield end

    
def confidence_interval(data):
    #using 95 percent confidence interval
    data_points = len(data)
    average_accuracy = 0
    for i in range(0,data_points):
        average_accuracy = average_accuracy + data[i]/data_points

    print("mean accuracy across k folds: " + str(average_accuracy))

    standard_deviation = 0
    variance = 0
    for i in range(0,data_points):
        variance = variance + ((data[i] - average_accuracy) * (data[i] - average_accuracy))
    
    standard_deviation = np.sqrt(variance/data_points)
    standard_error = standard_deviation/np.sqrt(data_points)

    print("confidence interval: " + str(average_accuracy - 1.96*standard_error) + ", " + str(average_accuracy + 1.96*standard_error))
    return [average_accuracy - 1.96*standard_error, average_accuracy, average_accuracy + 1.96*standard_error]

In [3]:
total_data_set = np.vstack((trX, teX))
total_label_set = np.vstack((trY, teY))

prediction = 0

average_model_accuracy = 0

models_to_train = 5

epochs_per_model = 3

batch_size = 128

number_of_folds = 10

average_model_accuracies = []
ensemble_method_accuracies = []
ensemble_method_time = []

#models_to_train_begin = 2
#models_to_train_end = 10

models_to_train_begin = 1
models_to_train_end = 1

confidence_interval_across_experiments = []
confidence_interval_across_experiments_time = []
aggregated_time_across_models = 0

#saver = tf.train.Saver()

# Launch the graph in a session
with tf.Session() as sess:
    for n in range(models_to_train_begin, models_to_train_end + 1):
        models_to_train = n
        average_model_accuracies = []
        ensemble_method_accuracies = []
        ensemble_method_time = []
        for fold in range(0, number_of_folds):
            print("fold number: " + str(fold))
            size_of_fold = int(len(total_data_set)/number_of_folds)
            trX_for_models = np.vstack((total_data_set[0:fold*size_of_fold], total_data_set[(fold + 1)*size_of_fold:len(total_data_set)]))
            teX = total_data_set[fold*size_of_fold:(fold + 1)*size_of_fold]

            trY_for_models = np.vstack((total_label_set[0:fold*size_of_fold], total_label_set[(fold + 1)*size_of_fold:len(total_label_set)]))
            teY = total_label_set[fold*size_of_fold:(fold + 1)*size_of_fold]

            print("x_train shape:", trX_for_models.shape, "y_train shape:", trY_for_models.shape)
            print("x_test shape:", teX.shape, "y_test shape:", teY.shape)

            average_model_accuracy = 0
            aggregated_time_across_models = 0

            for z in range(0,models_to_train):

                training_per_model = int(len(trX_for_models)/models_to_train)
                trX = trX_for_models[z*training_per_model:(z+1)*training_per_model]
                trY = trY_for_models[z*training_per_model:(z+1)*training_per_model]
                #print(trX.shape)
                #print(trY.shape)

                size_h1 = tf.constant(625, dtype=tf.int32)
                size_h2 = tf.constant(300, dtype=tf.int32)

                X = tf.placeholder("float", [None, 784])
                Y = tf.placeholder("float", [None, 10])

                w_h1 = init_weights([784, size_h1]) # create symbolic variables
                w_h2 = init_weights([size_h1, size_h2])
                w_o = init_weights([size_h2, 10])

                py_x = model(X, w_h1, w_h2, w_o)

                cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y)) # compute costs
                train_op = tf.train.GradientDescentOptimizer(0.05).minimize(cost) # construct an optimizer
                predict_op = tf.argmax(py_x, 1)
                tf.global_variables_initializer().run()
                for i in range(epochs_per_model):
                    time_start = time.process_time()
                    for start, end in zip(myRange(0, len(trX), batch_size), myRange(batch_size, len(trX)+1, batch_size)):
                        sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end]})
                    time_stop = time.process_time()
                    aggregated_time_across_models = aggregated_time_across_models + (time_stop - time_start)
                    print(i, np.mean(np.argmax(teY, axis=1) ==
                                     sess.run(predict_op, feed_dict={X: teX})))
                average_model_accuracy = np.mean(np.argmax(teY, axis=1) == sess.run(predict_op, feed_dict={X: teX}))/models_to_train + average_model_accuracy
                if z == 0:
                    prediction = sess.run(py_x, feed_dict={X: teX})
                else:
                    prediction = np.add(prediction, sess.run(py_x, feed_dict={X: teX}))

            print("average accuracy of all the models: " + str(average_model_accuracy))
            print("accuracy of the predictions of all the models combined: " + str(np.mean(np.argmax(teY, axis=1) == np.argmax(prediction, axis=1)))) 
            average_model_accuracies.append(average_model_accuracy)
            ensemble_method_accuracies.append(np.mean(np.argmax(teY, axis=1) == np.argmax(prediction, axis=1)))
            ensemble_method_time.append(aggregated_time_across_models)

        answer = confidence_interval(ensemble_method_accuracies)
        print(answer)
        confidence_interval_across_experiments.append(answer)
        
        answer = confidence_interval(ensemble_method_time)
        print(answer)
        confidence_interval_across_experiments_time.append(answer)

print(confidence_interval_across_experiments)
confidence_interval_across_experiments = np.asarray(confidence_interval_across_experiments)
#np.save("ensembleMethodResultsAccuracy.npy", confidence_interval_across_experiments)
np.save("normalMethodResultsTime.npy", confidence_interval_across_experiments_time)

print(confidence_interval_across_experiments_time)
confidence_interval_across_experiments_time = np.asarray(confidence_interval_across_experiments_time)
#np.save("ensembleMethodResultsTime.npy", confidence_interval_across_experiments_time)
np.save("normalMethodResultsTime.npy", confidence_interval_across_experiments_time)


fold number: 0
x_train shape: (63000, 784) y_train shape: (63000, 10)
x_test shape: (7000, 784) y_test shape: (7000, 10)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

0 0.4645714285714286
1 0.681
2 0.715
0 0.47828571428571426
1 0.6834285714285714
2 0.7275714285714285
average accuracy of all the models: 0.7212857142857143
accuracy of the predictions of all the models combined: 0.7421428571428571
fold number: 1
x_train shape: (63000, 784) y_train shape: (63000, 10)
x_test shape: (7000, 784) y_test shape: (7000, 10)
0 0.4561428571428571
1 0.6727142857142857
2 0.7094285714285714
0 0.4491428571428571
1 0.666
2 0.7195714285714285
average accuracy of all the models: 0.7144999999999999
accuracy of the predictions of all the models combined: 0.7261428571428571
fold number: 2
x_train shape: (63000, 784) y_train shape: (63000, 10)
x_test shape: (7000, 78

0 0.44857142857142857
1 0.5985714285714285
2 0.6875714285714286
0 0.2834285714285714
1 0.588
2 0.6562857142857143
0 0.3242857142857143
1 0.5485714285714286
2 0.5824285714285714
average accuracy of all the models: 0.642095238095238
accuracy of the predictions of all the models combined: 0.6694285714285715
mean accuracy across k folds: 0.6873285714285715
confidence interval: 0.6820782839697747, 0.6925788588873684
[0.6820782839697747, 0.6873285714285715, 0.6925788588873684]
mean accuracy across k folds: 33.775
confidence interval: 32.82528719719459, 34.724712802805406
[32.82528719719459, 33.775, 34.724712802805406]
fold number: 0
x_train shape: (63000, 784) y_train shape: (63000, 10)
x_test shape: (7000, 784) y_test shape: (7000, 10)
0 0.19614285714285715
1 0.44157142857142856
2 0.5532857142857143
0 0.3018571428571429
1 0.4602857142857143
2 0.6058571428571429
0 0.24042857142857144
1 0.407
2 0.6317142857142857
0 0.3152857142857143
1 0.454
2 0.5748571428571428
average accuracy of all the mo

1 0.5992857142857143
2 0.6234285714285714
0 0.30957142857142855
1 0.44742857142857145
2 0.5568571428571428
average accuracy of all the models: 0.5814285714285714
accuracy of the predictions of all the models combined: 0.6037142857142858
fold number: 5
x_train shape: (63000, 784) y_train shape: (63000, 10)
x_test shape: (7000, 784) y_test shape: (7000, 10)
0 0.30314285714285716
1 0.545
2 0.5682857142857143
0 0.1992857142857143
1 0.4257142857142857
2 0.5984285714285714
0 0.2844285714285714
1 0.43385714285714283
2 0.6435714285714286
0 0.2825714285714286
1 0.5642857142857143
2 0.674
0 0.32142857142857145
1 0.4644285714285714
2 0.5758571428571428
average accuracy of all the models: 0.6120285714285715
accuracy of the predictions of all the models combined: 0.6241428571428571
fold number: 6
x_train shape: (63000, 784) y_train shape: (63000, 10)
x_test shape: (7000, 784) y_test shape: (7000, 10)
0 0.371
1 0.5397142857142857
2 0.577
0 0.191
1 0.49857142857142855
2 0.596
0 0.2537142857142857
1 0

2 0.4937142857142857
0 0.272
1 0.2915714285714286
2 0.402
0 0.18642857142857142
1 0.39514285714285713
2 0.4055714285714286
0 0.27985714285714286
1 0.2887142857142857
2 0.5421428571428571
0 0.30228571428571427
1 0.30957142857142855
2 0.372
0 0.10028571428571428
1 0.28314285714285714
2 0.41114285714285714
average accuracy of all the models: 0.4377619047619048
accuracy of the predictions of all the models combined: 0.4554285714285714
fold number: 9
x_train shape: (63000, 784) y_train shape: (63000, 10)
x_test shape: (7000, 784) y_test shape: (7000, 10)
0 0.09657142857142857
1 0.26985714285714285
2 0.4522857142857143
0 0.278
1 0.2938571428571429
2 0.4115714285714286
0 0.18342857142857144
1 0.38385714285714284
2 0.42657142857142855
0 0.26271428571428573
1 0.2937142857142857
2 0.5501428571428572
0 0.3037142857142857
1 0.30942857142857144
2 0.35428571428571426
0 0.25585714285714284
1 0.2834285714285714
2 0.44742857142857145
average accuracy of all the models: 0.4403809523809524
accuracy of th

0 0.327
1 0.33585714285714285
2 0.478
0 0.19328571428571428
1 0.21671428571428572
2 0.4695714285714286
0 0.2267142857142857
1 0.3752857142857143
2 0.5752857142857143
0 0.271
1 0.3544285714285714
2 0.4662857142857143
0 0.31742857142857145
1 0.3482857142857143
2 0.49442857142857144
0 0.10142857142857142
1 0.3525714285714286
2 0.5498571428571428
0 0.20142857142857143
1 0.376
2 0.4717142857142857
0 0.22728571428571429
1 0.363
2 0.49514285714285716
average accuracy of all the models: 0.5000357142857143
accuracy of the predictions of all the models combined: 0.5538571428571428
fold number: 1
x_train shape: (63000, 784) y_train shape: (63000, 10)
x_test shape: (7000, 784) y_test shape: (7000, 10)
0 0.295
1 0.31042857142857144
2 0.4855714285714286
0 0.19328571428571428
1 0.22471428571428573
2 0.476
0 0.196
1 0.3995714285714286
2 0.5017142857142857
0 0.2507142857142857
1 0.35485714285714287
2 0.4614285714285714
0 0.319
1 0.3404285714285714
2 0.48228571428571426
0 0.09714285714285714
1 0.3381428

0 0.24514285714285713
1 0.2865714285714286
2 0.4155714285714286
0 0.32971428571428574
1 0.34914285714285714
2 0.40914285714285714
0 0.28
1 0.3171428571428571
2 0.4654285714285714
0 0.28085714285714286
1 0.42614285714285716
2 0.4585714285714286
0 0.19542857142857142
1 0.257
2 0.39085714285714285
0 0.24314285714285713
1 0.489
2 0.5331428571428571
0 0.10342857142857143
1 0.37085714285714283
2 0.4072857142857143
0 0.19628571428571429
1 0.31142857142857144
2 0.414
0 0.19528571428571428
1 0.30214285714285716
2 0.49214285714285716
average accuracy of all the models: 0.44290476190476186
accuracy of the predictions of all the models combined: 0.5204285714285715
fold number: 2
x_train shape: (63000, 784) y_train shape: (63000, 10)
x_test shape: (7000, 784) y_test shape: (7000, 10)
0 0.24057142857142857
1 0.2592857142857143
2 0.412
0 0.09714285714285714
1 0.38185714285714284
2 0.4452857142857143
0 0.194
1 0.40614285714285714
2 0.5124285714285715
0 0.27885714285714286
1 0.42557142857142854
2 0.454

2 0.36514285714285716
0 0.10042857142857142
1 0.41528571428571426
2 0.4928571428571429
0 0.196
1 0.21257142857142858
2 0.3427142857142857
0 0.09628571428571428
1 0.24242857142857144
2 0.3485714285714286
0 0.10042857142857142
1 0.32057142857142856
2 0.5257142857142857
0 0.18957142857142859
1 0.3605714285714286
2 0.366
0 0.19785714285714287
1 0.3232857142857143
2 0.5015714285714286
0 0.196
1 0.1972857142857143
2 0.30914285714285716
0 0.09942857142857142
1 0.26785714285714285
2 0.3465714285714286
0 0.18428571428571427
1 0.32971428571428574
2 0.4125714285714286
average accuracy of all the models: 0.4010857142857144
accuracy of the predictions of all the models combined: 0.5572857142857143
fold number: 2
x_train shape: (63000, 784) y_train shape: (63000, 10)
x_test shape: (7000, 784) y_test shape: (7000, 10)
0 0.31742857142857145
1 0.322
2 0.38185714285714284
0 0.18357142857142858
1 0.273
2 0.35328571428571426
0 0.202
1 0.22385714285714287
2 0.349
0 0.101
1 0.24428571428571427
2 0.367
0 0.0